In [ ]:

!pip -q install --force-reinstall --no-binary :all: numpy==1.26.4
!pip -q install --force-reinstall pandas==2.2.2 scipy==1.11.4 scikit-learn==1.4.2 plotly==5.24.1 cython==0.29.37

!pip -q install tpot==0.12.1

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 131.8 MB/s eta 0:00:00


In [ ]:

!pip -q install -U numpy==2.1.3 pandas==2.2.3 scikit-learn==1.6.1 plotly==5.24.1 requests==2.32.4

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-adk 1.16.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.3 which is incompatible.


{'status': 'ok', 'restart': True}

In [ ]:
import pandas as pd, numpy as np, plotly.express as px, io, warnings, time
import requests, re
from urllib.parse import quote

warnings.filterwarnings("ignore")

URL_HOSP_CANDIDATES = [
    "https://data.cms.gov/provider-data/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD",  # Hospital General Info
]
URL_DEATHS_CANDIDATES = [
    "https://data.cms.gov/provider-data/api/views/ynj2-r877/rows.csv?accessType=DOWNLOAD",  # Complications & Deaths (provider-level)
]
URL_MSPB_CANDIDATES = [
    "https://data.cms.gov/provider-data/api/views/rrqw-56er/rows.csv?accessType=DOWNLOAD",  # MSPB hospital-level
]


def ascii_only(url: str) -> str:

    url = (url.replace("…","...")
              .replace("“",'"').replace("”",'"')
              .replace("’","'").replace("–","-").replace("—","-"))
    try:
        url.encode("ascii")
        return url
    except UnicodeEncodeError:
        return quote(url, safe=":/?&=%._-")

def download_with_retry(url, tries=3, sleep=2):
    url = ascii_only(url)
    last = None
    for _ in range(tries):
        try:
            r = requests.get(url, timeout=60)
            r.raise_for_status()

            try:
                return pd.read_csv(io.StringIO(r.content.decode("utf-8")), dtype=str, low_memory=False)
            except UnicodeDecodeError:
                return pd.read_csv(io.StringIO(r.content.decode("latin-1")), dtype=str, low_memory=False)
        except Exception as e:
            last = e
            time.sleep(sleep)
    raise last

def safe_read_any(candidates, label="file"):
    errs = []
    for u in candidates:
        try:

            return pd.read_csv(ascii_only(u), dtype=str, low_memory=False, encoding="utf-8")
        except Exception:
            try:
                return download_with_retry(u)
            except Exception as e:
                print(f"[{label}] Failed: {u}\n  -> {e}")
                errs.append((u, str(e)))
    # Last resort: manual upload (keeps notebook running)
    try:
        from google.colab import files  # type: ignore
        print(f"All {label} URLs failed. Please upload the {label} CSV when prompted.")
        uploaded = files.upload()
        fname = list(uploaded.keys())[0]
        return pd.read_csv(fname, dtype=str, low_memory=False)
    except Exception as e:

        print(f"No upload provided. Proceeding with empty {label} DataFrame. You can re-run later.")
        return pd.DataFrame()

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    df.columns = (df.columns
                    .str.strip()
                    .str.lower()
                    .str.replace(r'[^a-z0-9]+','_', regex=True))
    return df

def to_numeric(df, cols):
    if df.empty: return df
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


hosp   = safe_read_any(URL_HOSP_CANDIDATES,  label="Hospital General Info")
deaths = safe_read_any(URL_DEATHS_CANDIDATES, label="Complications & Deaths")
mspb   = safe_read_any(URL_MSPB_CANDIDATES,   label="MSPB")

hosp, deaths, mspb = normalize_cols(hosp), normalize_cols(deaths), normalize_cols(mspb)


for df in (hosp, deaths, mspb):
    if df.empty: continue
    if "provider_id" in df.columns: df.rename(columns={"provider_id":"ccn"}, inplace=True)
    if "facility_id" in df.columns: df.rename(columns={"facility_id":"ccn"}, inplace=True)
    if "ccn" in df.columns:
        df["ccn"] = (df["ccn"].astype(str).str.strip()
                                .str.replace(r"\.0$","", regex=True)
                                .str.zfill(6))


if not hosp.empty:
    h_keep = ["ccn","hospital_name","address","city","state","zip_code","hospital_type",
              "hospital_ownership","emergency_services","overall_hospital_quality_star_rating"]
    hosp_small = hosp[[c for c in h_keep if c in hosp.columns]].drop_duplicates(subset=["ccn"]) if "ccn" in hosp.columns else hosp.copy()
    if "overall_hospital_quality_star_rating" in hosp_small.columns:
        hosp_small["overall_hospital_quality_star_rating"] = (
            hosp_small["overall_hospital_quality_star_rating"].replace({"Not Available": np.nan})
        )
        hosp_small["overall_hospital_quality_star_rating"] = pd.to_numeric(
            hosp_small["overall_hospital_quality_star_rating"], errors="coerce"
        )
else:
    hosp_small = pd.DataFrame()

if not deaths.empty:
    death_cols = [c for c in deaths.columns if (("30_day" in c and "death" in c) or "death_rate" in c or "mortality" in c)]
    if "ccn" in deaths.columns and death_cols:
        deaths_small = deaths[["ccn"] + death_cols].drop_duplicates()
    else:
        deaths_small = deaths.copy()
    deaths_small = to_numeric(deaths_small, [c for c in death_cols if c!="ccn"])
else:
    deaths_small, death_cols = pd.DataFrame(), []

if not mspb.empty:
    mspb_cols = [c for c in mspb.columns if ("mspb" in c or "spending" in c or "payment" in c)]
    if "ccn" in mspb.columns and mspb_cols:
        mspb_small = mspb[["ccn"] + mspb_cols].drop_duplicates()
    else:
        mspb_small = mspb.copy()
    mspb_small = to_numeric(mspb_small, [c for c in mspb_cols if c!="ccn"])
else:
    mspb_small, mspb_cols = pd.DataFrame(), []


df = hosp_small.copy()
if not df.empty and not deaths_small.empty and "ccn" in df.columns and "ccn" in deaths_small.columns:
    df = df.merge(deaths_small, on="ccn", how="left")
if not df.empty and not mspb_small.empty and "ccn" in df.columns and "ccn" in mspb_small.columns:
    df = df.merge(mspb_small, on="ccn", how="left")

print("Joined shape:", df.shape)
display(df.head(3) if not df.empty else pd.DataFrame({"info":["No rows to display yet. Check URLs or upload CSVs."]}))


if not df.empty and "overall_hospital_quality_star_rating" in df.columns:
    fig = px.histogram(df, x="overall_hospital_quality_star_rating", nbins=5, title="Hospital Star Ratings")
    fig.show()

mort_col  = next((c for c in df.columns if c not in ["ccn"] and ("mortality" in c or "death" in c)), None)
spend_col = next((c for c in df.columns if c not in ["ccn"] and ("mspb" in c or "spending" in c or "payment" in c)), None)

if not df.empty and mort_col and spend_col and "state" in df.columns:
    fig = px.scatter(df, x=spend_col, y=mort_col, color="state",
                     hover_data=[c for c in ["hospital_name","ccn"] if c in df.columns],
                     title=f"Spending vs Mortality proxy ({spend_col} vs {mort_col})")
    fig.show()


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

if not df.empty and "overall_hospital_quality_star_rating" in df.columns and "state" in df.columns:
    work = df.dropna(subset=["overall_hospital_quality_star_rating","state"]).copy()
    if not work.empty:
        work["high_star"] = (work["overall_hospital_quality_star_rating"] >= 4).astype(int)
        feats = [c for c in [spend_col, mort_col] if c]
        if feats:
            X = work[feats + ["state"]].copy()
            y = work["high_star"].copy()
            pre = ColumnTransformer(
                transformers=[
                    ("num","passthrough", feats),
                    ("cat", OneHotEncoder(handle_unknown="ignore"), ["state"])
                ]
            )
            pipe = Pipeline([("pre", pre), ("lr", LogisticRegression(max_iter=2000))])
            Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
            pipe.fit(Xtr, ytr)
            proba = pipe.predict_proba(Xte)[:,1]
            print("Baseline AUC:", roc_auc_score(yte, proba))
            print(classification_report(yte, (proba>0.5).astype(int)))
        else:
            print("Model step skipped: no numeric spend/mortality columns found.")
    else:
        print("Model step skipped: insufficient non-null rows.")


keep = [c for c in ["ccn","hospital_name","state","overall_hospital_quality_star_rating", spend_col, mort_col] if c and c in df.columns]
if keep:
    dash = df[keep].dropna()
    dash.columns = [c.title().replace("_"," ") for c in dash.columns]
else:

    dash = pd.DataFrame(columns=["Ccn","Hospital Name","State","Overall Hospital Quality Star Rating","Spend","Mortality"])

dash.to_csv("hospital_dash.csv", index=False)
print("Saved: hospital_dash.csv  → You can upload to Google Sheets → Connect in Looker Studio.")

if not df.empty and "state" in df.columns and "overall_hospital_quality_star_rating" in df.columns:
    top = (df.dropna(subset=["overall_hospital_quality_star_rating"])
             .groupby("state", as_index=False)["overall_hospital_quality_star_rating"]
             .mean().sort_values("overall_hospital_quality_star_rating", ascending=False)
             .head(10))
    display(top.rename(columns={"overall_hospital_quality_star_rating":"Avg Stars (Top 10 States)"}))
else:
    print("Leaderboard skipped: missing state or star rating.")


[Hospital General Info] Failed: https://data.cms.gov/provider-data/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD
  -> 404 Client Error: Not Found for url: https://data.cms.gov/provider-data/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD
All Hospital General Info URLs failed. Please upload the Hospital General Info CSV when prompted.


Saving Hospital_General_Information.csv to Hospital_General_Information.csv
[Complications & Deaths] Failed: https://data.cms.gov/provider-data/api/views/ynj2-r877/rows.csv?accessType=DOWNLOAD
  -> 404 Client Error: Not Found for url: https://data.cms.gov/provider-data/api/views/ynj2-r877/rows.csv?accessType=DOWNLOAD
All Complications & Deaths URLs failed. Please upload the Complications & Deaths CSV when prompted.


Saving Complications_and_Deaths-Hospital.csv to Complications_and_Deaths-Hospital.csv
[MSPB] Failed: https://data.cms.gov/provider-data/api/views/rrqw-56er/rows.csv?accessType=DOWNLOAD
  -> 404 Client Error: Not Found for url: https://data.cms.gov/provider-data/api/views/rrqw-56er/rows.csv?accessType=DOWNLOAD
All MSPB URLs failed. Please upload the MSPB CSV when prompted.


Saving Medicare_Hospital_Spending_Per_Patient-Hospital.csv to Medicare_Hospital_Spending_Per_Patient-Hospital.csv
Joined shape: (95726, 37)


,ccn,address_x,state_x,zip_code_x,hospital_type,hospital_ownership,emergency_services,facility_name_x,address_y,city_town_x,...,state,zip_code,county_parish_y,telephone_number_y,measure_id_y,measure_name_y,score_y,footnote_y,start_date_y,end_date_y
0,010001,1108 ROSS CLARK CIRCLE,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,Yes,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,...,AL,36301,HOUSTON,(334) 793-8701,MSPB-1,Medicare hospital spending per patient (Medica...,1.03,NaN,01/01/2023,12/31/2023
1,010001,1108 ROSS CLARK CIRCLE,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,Yes,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,...,AL,36301,HOUSTON,(334) 793-8701,MSPB-1,Medicare hospital spending per patient (Medica...,1.03,NaN,01/01/2023,12/31/2023
2,010001,1108 ROSS CLARK CIRCLE,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,Yes,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,...,AL,36301,HOUSTON,(334) 793-8701,MSPB-1,Medicare hospital spending per patient (Medica...,1.03,NaN,01/01/2023,12/31/2023


Saved: hospital_dash.csv  → You can upload to Google Sheets → Connect in Looker Studio.
Leaderboard skipped: missing state or star rating.


In [ ]:
from google.colab import files

try:
    files.download("hospital_dash.csv")
    print("Download started: hospital_dash.csv")
except Exception as e:
    print("Download failed:", e)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started: hospital_dash.csv


In [ ]:

base_cols = [
    "ccn","hospital_name","address","city","state","zip_code",
    "hospital_type","hospital_ownership","emergency_services",
    "overall_hospital_quality_star_rating"
]

# Pull a handful of relevant measure columns if present
mortality_cols = [c for c in df.columns
                  if ("mortality" in c or "death" in c) and c != "ccn"][:8]
readmit_cols   = [c for c in df.columns if "readmission" in c][:8]
spend_cols     = [c for c in df.columns
                  if ("mspb" in c or "spending" in c or "payment" in c) and c != "ccn"][:8]

# Build the final column list (only those that actually exist)
keep = [c for c in base_cols if c in df.columns] \
     + [c for c in mortality_cols if c in df.columns] \
     + [c for c in readmit_cols if c in df.columns] \
     + [c for c in spend_cols if c in df.columns]


for c in (spend_col, mort_col):
    if c and c in df.columns and c not in keep:
        keep.append(c)


dash = df[keep].copy()

rename_map = {
    "ccn": "CCN",
    "hospital_name": "Hospital",
    "address": "Address",
    "city": "City",
    "state": "State",
    "zip_code": "ZIP",
    "hospital_type": "Hospital Type",
    "hospital_ownership": "Ownership",
    "emergency_services": "ED Services",
    "overall_hospital_quality_star_rating": "Stars"
}
dash.rename(columns=rename_map, inplace=True)


sort_cols = [c for c in ["State","Stars","Hospital"] if c in dash.columns]
if sort_cols:
    dash.sort_values(sort_cols, ascending=[True, False, True][:len(sort_cols)], inplace=True)

# Write CSV
dash.to_csv("hospital_dash.csv", index=False)
print(f"Saved with {dash.shape[0]} rows and {dash.shape[1]} columns: hospital_dash.csv")

# Offer download in Colab
try:
    from google.colab import files
    files.download("hospital_dash.csv")
    print("Download started: hospital_dash.csv")
except Exception as e:
    print("Download prompt not available in this environment:", e)


Saved with 95726 rows and 7 columns: hospital_dash.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started: hospital_dash.csv


In [ ]:

base_cols = [
    "ccn", "hospital_name", "address", "city", "state", "zip_code",
    "hospital_type", "hospital_ownership", "emergency_services",
    "overall_hospital_quality_star_rating"
]

metric_patterns = ["mortality", "death", "readmission", "complication", "spending", "mspb", "payment"]
metric_cols = [c for c in df.columns if any(p in c.lower() for p in metric_patterns)]

keep = [c for c in base_cols + metric_cols if c in df.columns]


dash = df[keep].copy()


dash.columns = [c.replace("_", " ").title() for c in dash.columns]

# Sort by state + hospital name if available
sort_cols = [c for c in ["State", "Hospital Name"] if c in dash.columns]
if sort_cols:
    dash.sort_values(sort_cols, inplace=True)

# Save to CSV
dash.to_csv("hospital_dash.csv", index=False)
print(f"✅ Saved hospital_dash.csv with {dash.shape[0]} rows and {dash.shape[1]} columns.")

# Offer download in Colab
try:
    from google.colab import files
    files.download("hospital_dash.csv")
    print("Download started: hospital_dash.csv")
except Exception as e:
    print("Download prompt unavailable:", e)


✅ Saved hospital_dash.csv with 95726 rows and 7 columns.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started: hospital_dash.csv


In [ ]:
# Colab-friendly pins: matches Colab's stack and avoids resolver drama
!pip -q install -U numpy==2.1.3 pandas==2.2.3 scikit-learn==1.6.1 plotly==5.24.1 requests==2.32.4

import IPython
IPython.Application.instance().kernel.do_shutdown(True)  # auto-restart so binaries load cleanly


{'status': 'ok', 'restart': True}

In [ ]:
import pandas as pd, numpy as np, plotly.express as px, io, warnings, time, re
import requests
from urllib.parse import quote

warnings.filterwarnings("ignore")


URL_HOSP_CANDIDATES = [
    # Hospital General Information
    "https://data.cms.gov/provider-data/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD",
]
URL_DEATHS_CANDIDATES = [
    # Complications & Deaths (hospital-level)
    "https://data.cms.gov/provider-data/api/views/ynj2-r877/rows.csv?accessType=DOWNLOAD",
]
URL_MSPB_CANDIDATES = [
    # Medicare Spending per Beneficiary (Hospital)
    "https://data.cms.gov/provider-data/api/views/rrqw-56er/rows.csv?accessType=DOWNLOAD",
]

def ascii_only(url: str) -> str:
    url = (url.replace("…","...")
              .replace("“",'"').replace("”",'"')
              .replace("’","'").replace("–","-").replace("—","-"))
    try:
        url.encode("ascii")
        return url
    except UnicodeEncodeError:
        return quote(url, safe=":/?&=%._-")

def download_with_retry(url, tries=3, sleep=2):
    url = ascii_only(url)
    last = None
    for _ in range(tries):
        try:
            r = requests.get(url, timeout=60)
            r.raise_for_status()
            try:
                return pd.read_csv(io.StringIO(r.content.decode("utf-8")), dtype=str, low_memory=False)
            except UnicodeDecodeError:
                return pd.read_csv(io.StringIO(r.content.decode("latin-1")), dtype=str, low_memory=False)
        except Exception as e:
            last = e
            time.sleep(sleep)
    raise last

def safe_read_any(candidates, label="file"):
    errs = []
    for u in candidates:
        try:
            return pd.read_csv(ascii_only(u), dtype=str, low_memory=False, encoding="utf-8")
        except Exception:
            try:
                return download_with_retry(u)
            except Exception as e:
                print(f"[{label}] Failed: {u}\n  -> {e}")
                errs.append((u, str(e)))
    # Upload fallback (keeps notebook moving)
    try:
        from google.colab import files  # type: ignore
        print(f"All {label} URLs failed. Please upload the {label} CSV when prompted.")
        uploaded = files.upload()
        fname = list(uploaded.keys())[0]
        return pd.read_csv(fname, dtype=str, low_memory=False)
    except Exception:
        print(f"No upload provided for {label}. Returning empty DataFrame.")
        return pd.DataFrame()

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    df.columns = (df.columns
                    .str.strip()
                    .str.lower()
                    .str.replace(r'[^a-z0-9]+','_', regex=True))
    return df


hosp   = safe_read_any(URL_HOSP_CANDIDATES,  label="Hospital General Info")
deaths = safe_read_any(URL_DEATHS_CANDIDATES, label="Complications & Deaths")
mspb   = safe_read_any(URL_MSPB_CANDIDATES,   label="MSPB")

hosp, deaths, mspb = normalize_cols(hosp), normalize_cols(deaths), normalize_cols(mspb)


for df in (hosp, deaths, mspb):
    if df.empty: continue
    if "provider_id" in df.columns: df.rename(columns={"provider_id":"ccn"}, inplace=True)
    if "facility_id" in df.columns: df.rename(columns={"facility_id":"ccn"}, inplace=True)
    if "ccn" in df.columns:
        df["ccn"] = (df["ccn"].astype(str).str.strip()
                                .str.replace(r"\.0$","", regex=True)
                                .str.zfill(6))


def clean_measure_label(s: str) -> str:
    if not isinstance(s, str): return s
    s = s.strip().replace("%","pct").replace("/","_").replace("-","_")
    s = re.sub(r'[^A-Za-z0-9_]+','_', s)
    s = re.sub(r'_+','_', s).strip('_').lower()
    return s

def pick_value_col(df, candidates=("score","value","rate","mspb_amount","mspb_value","amount","numerator")):
    for c in candidates:
        if c in df.columns: return c
    # fallback: any numeric-ish column
    for c in df.columns:
        if c != "ccn" and df[c].dropna().astype(str).str.match(r"^-?\d+(\.\d+)?$").any():
            return c
    return None

def reshape_measures_long_to_wide(df, id_col="ccn"):
    if df.empty or id_col not in df.columns:
        return pd.DataFrame(columns=[id_col])
    measure_id = "measure_id" if "measure_id" in df.columns else None
    measure_name = "measure_name" if "measure_name" in df.columns else None
    value_col = pick_value_col(df)
    if (measure_id or measure_name) and value_col:
        tmp = df[[id_col] + [c for c in [measure_id, measure_name, value_col] if c]].copy()
        tmp[value_col] = pd.to_numeric(tmp[value_col], errors="coerce")
        tmp = tmp.dropna(subset=[value_col])
        if measure_id and measure_name:
            tmp["col"] = np.where(tmp[measure_id].notna() & tmp[measure_id].astype(str).str.len()>0,
                                  tmp[measure_id].astype(str), tmp[measure_name].astype(str))
        elif measure_id:
            tmp["col"] = tmp[measure_id].astype(str)
        else:
            tmp["col"] = tmp[measure_name].astype(str)
        tmp["col"] = tmp["col"].map(clean_measure_label)
        wide = tmp.pivot_table(index=id_col, columns="col", values=value_col, aggfunc="mean").reset_index()
        wide.columns = [id_col] + [clean_measure_label(c) for c in wide.columns.tolist()[1:]]
        return wide
    # already wide or unrecognized schema
    return df

deaths_wide = reshape_measures_long_to_wide(deaths, id_col="ccn")
mspb_wide   = reshape_measures_long_to_wide(mspb,   id_col="ccn")

if "ccn" in deaths_wide.columns:
    deaths_wide = deaths_wide.groupby("ccn", as_index=False).first()
if "ccn" in mspb_wide.columns:
    mspb_wide = mspb_wide.groupby("ccn", as_index=False).first()


base_cols = [
    "ccn","hospital_name","address","city","state","zip_code",
    "hospital_type","hospital_ownership","emergency_services",
    "overall_hospital_quality_star_rating"
]
hosp_small = hosp[[c for c in base_cols if c in hosp.columns]].drop_duplicates(subset=["ccn"]) if not hosp.empty else pd.DataFrame()

if "overall_hospital_quality_star_rating" in hosp_small.columns:
    hosp_small["overall_hospital_quality_star_rating"] = pd.to_numeric(
        hosp_small["overall_hospital_quality_star_rating"].replace({"Not Available": np.nan}),
        errors="coerce"
    )


df = hosp_small.copy()
if not deaths_wide.empty and "ccn" in deaths_wide.columns:
    df = df.merge(deaths_wide, on="ccn", how="left")
if not mspb_wide.empty and "ccn" in mspb_wide.columns:
    df = df.merge(mspb_wide, on="ccn", how="left")

print("Joined shape:", df.shape)
display(df.head(3) if not df.empty else pd.DataFrame({"info":["No rows yet — check URLs or upload CSVs."]}))


metric_patterns = ["mort", "death", "readmit", "complication", "mspb", "spend", "payment", "rate", "score"]
metric_cols = [c for c in df.columns if any(p in c.lower() for p in metric_patterns)]

export_cols = [c for c in base_cols if c in df.columns] + [c for c in metric_cols if c in df.columns]
dash = df[export_cols].copy()


header_map = {
    "ccn": "CCN",
    "hospital_name": "Hospital",
    "address": "Address",
    "city": "City",
    "state": "State",
    "zip_code": "ZIP",
    "hospital_type": "Hospital Type",
    "hospital_ownership": "Ownership",
    "emergency_services": "ED Services",
    "overall_hospital_quality_star_rating": "Stars",
}
dash.rename(columns=header_map, inplace=True)
dash.columns = [c if c in header_map.values() else c.replace("_"," ").title() for c in dash.columns]


for_sort = [c for c in ["State","Stars","Hospital"] if c in dash.columns]
if for_sort:
    dash.sort_values(for_sort, ascending=[True, False, True][:len(for_sort)], inplace=True)


dash.to_csv("hospital_dash.csv", index=False)
print(f"✅ Saved hospital_dash.csv with {dash.shape[0]} rows and {dash.shape[1]} columns.")

try:
    from google.colab import files
    files.download("hospital_dash.csv")
    print("Download started: hospital_dash.csv")
except Exception as e:
    print("Download prompt unavailable:", e)

print("Sample columns:", dash.columns[:25].tolist())


[Hospital General Info] Failed: https://data.cms.gov/provider-data/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD
  -> 404 Client Error: Not Found for url: https://data.cms.gov/provider-data/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD
All Hospital General Info URLs failed. Please upload the Hospital General Info CSV when prompted.


Saving Hospital_General_Information.csv to Hospital_General_Information (2).csv
[Complications & Deaths] Failed: https://data.cms.gov/provider-data/api/views/ynj2-r877/rows.csv?accessType=DOWNLOAD
  -> 404 Client Error: Not Found for url: https://data.cms.gov/provider-data/api/views/ynj2-r877/rows.csv?accessType=DOWNLOAD
All Complications & Deaths URLs failed. Please upload the Complications & Deaths CSV when prompted.


Saving Complications_and_Deaths-Hospital.csv to Complications_and_Deaths-Hospital (3).csv
[MSPB] Failed: https://data.cms.gov/provider-data/api/views/rrqw-56er/rows.csv?accessType=DOWNLOAD
  -> 404 Client Error: Not Found for url: https://data.cms.gov/provider-data/api/views/rrqw-56er/rows.csv?accessType=DOWNLOAD
All MSPB URLs failed. Please upload the MSPB CSV when prompted.


Saving Medicare_Hospital_Spending_Per_Patient-Hospital.csv to Medicare_Hospital_Spending_Per_Patient-Hospital (1).csv
Joined shape: (5381, 28)


,ccn,address,state,zip_code,hospital_type,hospital_ownership,emergency_services,abdominopelvic_accidental_puncture_or_laceration_rate,cms_medicare_psi_90_patient_safety_and_adverse_events_composite,comp_hip_knee,...,mort_30_ami,mort_30_stk,perioperative_pulmonary_embolism_or_deep_vein_thrombosis_rate,postoperative_acute_kidney_injury_requiring_dialysis_rate,postoperative_hemorrhage_or_hematoma_rate,postoperative_respiratory_failure_rate,postoperative_sepsis_rate,postoperative_wound_dehiscence_rate,pressure_ulcer_rate,medicare_hospital_spending_per_patient_medicare_spending_per_beneficiary
0,010001,1108 ROSS CLARK CIRCLE,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,Yes,1.37,0.98,3.2,...,11.4,13.5,6.08,1.36,2.85,12.33,5.00,1.63,0.11,1.03
1,010005,2505 U S HIGHWAY 431 NORTH,AL,35957,Acute Care Hospitals,Government - Hospital District or Authority,Yes,0.77,1.01,3.0,...,NaN,12.9,4.61,1.38,2.74,10.11,5.70,1.79,0.63,1.00
2,010006,1701 VETERANS DRIVE,AL,35630,Acute Care Hospitals,Proprietary,Yes,0.91,1.16,4.7,...,14.5,12.4,4.14,1.42,5.53,14.86,4.04,2.06,0.78,1.02


✅ Saved hospital_dash.csv with 5381 rows and 26 columns.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started: hospital_dash.csv
Sample columns: ['CCN', 'Address', 'State', 'ZIP', 'Hospital Type', 'Ownership', 'ED Services', 'Abdominopelvic Accidental Puncture Or Laceration Rate', 'Death Rate Among Surgical Inpatients With Serious Treatable Complications', 'Death Rate For Cabg Surgery Patients', 'Death Rate For Copd Patients', 'Death Rate For Heart Failure Patients', 'Death Rate For Pneumonia Patients', 'Hybrid Hospital Wide All Cause Risk Standardized Mortality Rate', 'Iatrogenic Pneumothorax Rate', 'In Hospital Fall Associated Fracture Rate', 'Mort 30 Ami', 'Mort 30 Stk', 'Perioperative Pulmonary Embolism Or Deep Vein Thrombosis Rate', 'Postoperative Acute Kidney Injury Requiring Dialysis Rate', 'Postoperative Hemorrhage Or Hematoma Rate', 'Postoperative Respiratory Failure Rate', 'Postoperative Sepsis Rate', 'Postoperative Wound Dehiscence Rate', 'Pressure Ulcer Rate']
